In [1]:
import streamlit as st
from google.oauth2 import service_account
from google.cloud import bigquery
import pandas as pd
import pandas_gbq as pdb
import tqdm

# Create API client.
credentials = service_account.Credentials.from_service_account_info(
    st.secrets["gcp_service_account"]
)
client = bigquery.Client(credentials=credentials)


In [2]:
# ### Download das bases
# #Dicionario
# df_dicionario = pdb.read_gbq("select * from basedosdados.br_ibge_pnad_covid.dicionario", project_id="postech-414800", credentials=credentials)
# df_dicionario.to_csv('.\\bases\\dicionario.csv',sep=";", header=True)

# #microdados
# df_ano_mes = pdb.read_gbq("select ano, mes, count(*) as cnt from basedosdados.br_ibge_pnad_covid.microdados group by ano, mes", project_id="postech-414800", credentials=credentials)
# df_ano_mes.sort_values(by=['mes'], inplace=True)

# lst_df = []
# for k,i in df_ano_mes.iterrows():
#     print(f"{i['ano']} - {i['mes']}")
#     df_tmp = pdb.read_gbq(f"select * from basedosdados.br_ibge_pnad_covid.microdados where ano = {i['ano']} and mes = {i['mes']}", project_id="postech-414800", credentials=credentials)
#     lst_df.append(df_tmp)

# df_concat = pd.concat(lst_df)
# df_concat.to_csv('.\\bases\\microdados.csv',sep=";", header=True)

In [3]:
df_dic = pd.read_csv(r'.\bases\dicionario.csv', sep=";")
df_m = pd.read_csv(r'.\bases\microdados.csv', sep=";")
df_desc = pd.read_csv(r'.\bases\desc_campos.csv', sep=";")

df_m.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
#fazer loop pelos campos
#atribuir o 'valor' ao nome do campo - ex df['capital'] = df['valor']
df_m_trat = df_m
for c in df_dic['nome_coluna'].unique():
    print(c)
    df_d = df_dic.query(f"nome_coluna == '{c}'")[['chave','valor']]
    df_m_trat = df_m_trat.merge(df_d, how='left', left_on=c, right_on='chave')
    df_m_trat[c] = df_m_trat['valor']
    df_m_trat.drop(columns=['chave','valor'], inplace=True)


capital
rm_ride
v1022
v1023
a001a
a003
a004
a005
a006
a006a
a006b
a007
a007a
a008
a009
b0011
b0012
b0013
b0014
b0015
b0016
b0017
b0018
b0019
b00110
b00111
b00112
b00113
b002
b0031
b0032
b0033
b0034
b0035
b0036
b0037
b0041
b0042
b0043
b0044
b0045
b0046
b005
b006
b007
b008
b009a
b009b
b009c
b009d
b009e
b009f
b0101
b0102
b0103
b0104
b0105
b0106
b011
c001
c002
c003
c004
c005
c006
c007
c007a
c007b
c007c
c007d
c007e
c007f
c009a
c010
c0101
c01011
c0102
c01021
c0103
c0104
c011a
c011a1
c011a11
c011a2
c011a21
c012
c013
c014
c015
c016
c017a
d0011
d0021
d0031
d0041
d0051
d0061
d0071
e001
e0021
e0022
e0023
e0024
f001
f0022
f002a1
f002a2
f002a3
f002a4
f002a5
f0061


In [9]:
estados = {
    "AC": "Acre",
    "AL": "Alagoas",
    "AP": "Amapá",
    "AM": "Amazonas",
    "BA": "Bahia",
    "CE": "Ceará",
    "DF": "Distrito Federal",
    "ES": "Espírito Santo",
    "GO": "Goiás",
    "MA": "Maranhão",
    "MT": "Mato Grosso",
    "MS": "Mato Grosso do Sul",
    "MG": "Minas Gerais",
    "PA": "Pará",
    "PB": "Paraíba",
    "PR": "Paraná",
    "PE": "Pernambuco",
    "PI": "Piauí",
    "RJ": "Rio de Janeiro",
    "RN": "Rio Grande do Norte",
    "RS": "Rio Grande do Sul",
    "RO": "Rondônia",
    "RR": "Roraima",
    "SC": "Santa Catarina",
    "SP": "São Paulo",
    "SE": "Sergipe",
    "TO": "Tocantins"
}

# Crie uma nova coluna com o nome dos estados por extenso
df_m_trat["estado"] = df_m_trat["sigla_uf"].map(estados)

In [13]:
df_m_trat.to_parquet(r'.\bases\microdados.parquet', index=False)

In [6]:
# #rename dos campos
# dct = df_desc[['fullname','description']].set_index('fullname').to_dict().get('description')
# df_m_trat.rename(columns=dct, inplace=True)